In [ ]:
# Librerias
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.mask import mask
from shapely.geometry import box
from joblib import load
from rasterio.features import shapes
from shapely.geometry import shape

In [ ]:
# === Cargar modelo ===
clf = load(fr"{modelo}.joblib")


# === Cargar imagen a clasificar ===
img_path = "L15-0585E-1041N_clip_7.tif"
with rasterio.open(img_path) as src:
    image = src.read()  # (bands, height, width)
    profile = src.profile
    transform = src.transform
    crs = src.crs

# Reorganizar
bands, height, width = image.shape
X_pred = image.reshape(bands, -1).T  # (n_pixels, n_bandas)

# Clasificar
y_pred = clf.predict(X_pred)
classification = y_pred.reshape((height, width))

# Guardar raster clasificado
nombredeclasificado='ClasificadoB'

profile.update(dtype=rasterio.uint8, count=1)
with rasterio.open(fr"{nombredeclasificado}.tif", "w", **profile) as dst:
    dst.write(classification.astype(np.uint8), 1)

print(fr"Clasificación guardada como: {nombredeclasificado}")

# Crear máscara para clase 1
mask = classification == 1

# Extraer polígonos
results = (
    {"properties": {"class": 1}, "geometry": shape(s)}
    for s, v in shapes(classification.astype(np.uint8), mask=mask, transform=transform)
)

# Crear GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(results, crs=crs)

# (Opcional) filtrar por área mínima
gdf_m = gdf.to_crs(epsg=3857)
gdf_m["Área [Ha]"] = gdf_m.area/10000
gdf_filtered = gdf_m[gdf_m["Área [Ha]"] >= 0.3].to_crs(crs)

# Exportar shapefile

gdf_filtered.to_file(fr"Resultados/{nombredeclasificado}.shp")

print(fr"Shapefile exportado como '{nombredeclasificado}.shp'")
